## Get the LFTest Corpus

In [1]:
# # If necessary:
# import os
# os.remove('snorkel.db')

# %load_ext autoreload
# %autoreload 2
# %matplotlib inline

# from snorkel import SnorkelSession
# session = SnorkelSession()

# from snorkel.parser import CorpusParser
# from snorkel.parser import HTMLParser
# from snorkel.parser import OmniParser

# path = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/lftest_html/'
# doc_parser = HTMLParser(path=path)
# context_parser = OmniParser()
# cp = CorpusParser(doc_parser, context_parser, max_docs=15) 

# %time corpus = cp.parse_corpus(name='Hardware', session=session)

In [2]:
# Verify documents
# for doc in corpus.documents: print doc

In [3]:
# session.add(corpus)
# session.commit()

## Load the Corpus and split into Train/Dev/Test

In [4]:
# from snorkel.models import Corpus

# corpus = session.query(Corpus).filter(Corpus.name == 'Hardware').one()
# print "%s contains %d Documents" % (corpus, len(corpus))


# from snorkel.utils import get_ORM_instance
# from snorkel.queries import split_corpus

# corpus = get_ORM_instance(Corpus, session, 'Hardware')
# split_corpus(session, corpus, train=0.8, development=0.2, test=0, seed=None)

# from snorkel.utils import get_ORM_instance
# from snorkel.models import Corpus

# corpus = get_ORM_instance(Corpus, session, 'Hardware Training')
# print "%s contains %d Documents" % (corpus, len(corpus))

# corpus = get_ORM_instance(Corpus, session, 'Hardware Development')
# print "%s contains %d Documents" % (corpus, len(corpus))

In [5]:
# # If necessary
# import os
# os.system('cp snorkel.db snorkel.db\ lftest_corpus');

In [6]:
import os
os.remove('snorkel.db');
os.system('cp snorkel.db\ corpus snorkel.db');


%load_ext autoreload
%autoreload 2

from snorkel import SnorkelSession
session = SnorkelSession()

from snorkel.models import Corpus
from snorkel.utils import get_ORM_instance

corpus = get_ORM_instance(Corpus, session, 'Hardware Training')
print "%s contains %d Documents" % (corpus, len(corpus))

from snorkel.models import candidate_subclass
from snorkel.matchers import RegexMatchSpan

Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

eeca_matcher = RegexMatchSpan(rgx='([b]{1}[abcdefklnpqruyz]{1}[\swxyz]?[0-9]{3,5}[\s]?[A-Z\/]{0,5}[0-9]?[A-Z]?([-][A-Z0-9]{1,7})?([-][A-Z0-9]{1,2})?)')
jedec_matcher = RegexMatchSpan(rgx='([123]N\d{3,4}[A-Z]{0,5}[0-9]?[A-Z]?)')
jis_matcher = RegexMatchSpan(rgx='(2S[abcdefghjkmqrstvz]{1}[\d]{2,4})')
others_matcher = RegexMatchSpan(rgx='((NSVBC|SMBT|MJ|MJE|MPS|MRF|RCA|TIP|ZTX|ZT|TIS|TIPL|DTC|MMBT|PZT){1}[\d]{2,4}[A-Z]{0,3}([-][A-Z0-9]{0,6})?([-][A-Z0-9]{0,1})?)')
parts_matcher = Union(eeca_matcher, jedec_matcher, jis_matcher, others_matcher)



temp_matcher = RegexMatchSpan(rgx=r'-[5-7][05]', longest_match_only=False)

import os
from hardware_utils import get_gold_dict
from collections import defaultdict

gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
gold_parts = get_gold_dict(gold_file, doc_on=True, part_on=True, val_on=False)
gold_parts_by_doc = defaultdict(set)
for part in gold_parts:
    gold_parts_by_doc[part[0]].add(part[1])
    
from hardware_utils import OmniNgramsPart, OmniNgramsTemp

part_ngrams = OmniNgramsPart(parts_by_doc=gold_parts_by_doc, n_max=3, split_tokens=None)
temp_ngrams = OmniNgramsTemp(n_max=3, split_tokens=None)

from snorkel.candidates import CandidateExtractor

ce = CandidateExtractor(Part_Temp, [part_ngrams, temp_ngrams], [parts_matcher, temp_matcher])

%time train = ce.extract(corpus.documents, 'Hardware Training Candidates', session)
print "%s contains %d Candidates" % (train, len(train))

session.add(train)
session.commit()

import os
os.system('cp snorkel.db snorkel.db\ lftest_candidates');

Corpus (Hardware Training) contains 75 Documents


NameError: name 'Union' is not defined

In [ ]:
# # If necessary:
# import os
# os.remove('snorkel.db');
# os.system('cp snorkel.db\ lftest_candidates snorkel.db');

# %load_ext autoreload
# %autoreload 2

# from snorkel import SnorkelSession
# session = SnorkelSession()

# from snorkel.models import candidate_subclass

# Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

# from snorkel.models import CandidateSet
# candidates = session.query(CandidateSet).filter(
#     CandidateSet.name == 'Hardware Training Candidates').one()
# print "%s contains %d Candidates" % (candidates, len(candidates))

# from snorkel.loaders import ExternalAnnotationsLoader

# loader = ExternalAnnotationsLoader(session, Part_Temp, 
#                                    'Hardware Training Candidates -- Gold',
#                                    'Hardware Training Labels -- Gold',
#                                    expand_candidate_set=True)

# import os
# from hardware_utils import load_hardware_labels

# filename = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
# %time load_hardware_labels(loader, candidates, filename, ['part','temp'], gold_attrib='stg_temp_min')

# from snorkel.models import Label

# train = session.query(CandidateSet).filter(
#     CandidateSet.name == 'Hardware Training Candidates').one()
# train_gold = session.query(CandidateSet).filter(
#     CandidateSet.name == 'Hardware Training Candidates -- Gold').one()
# print "%d/%d Candidates have positive Labels" % (len(train_gold), len(train))
# print "%d Labels loaded" % session.query(Label).filter(
#     Label.key == loader.annotation_key).count()

# for set_name in ['Development']:
#     candidate_set_name = 'Hardware %s Candidates' % set_name
#     candidates = session.query(CandidateSet).filter(
#         CandidateSet.name == candidate_set_name).one()
#     loader = ExternalAnnotationsLoader(session, Part_Temp, 
#                                        'Hardware %s Candidates -- Gold' % set_name,
#                                        'Hardware %s Labels -- Gold' % set_name,
#                                        expand_candidate_set=True)
#     %time load_hardware_labels(loader, candidates, filename, ['part','temp'], gold_attrib='stg_temp_min')
#     candidates_gold = session.query(CandidateSet).filter(
#         CandidateSet.name == candidate_set_name + ' -- Gold').one()
#     print "%d/%d Candidates in %s have positive Labels" % (
#         len(candidates_gold), len(candidates), candidates)
#     print "%d Labels loaded" % session.query(Label).filter(
#         Label.key == loader.annotation_key).count()
    
# # If necessary
# import os
# os.system('cp snorkel.db snorkel.db\ lftest_labels');

In [ ]:
# # If necessary:
# import os
# os.remove('snorkel.db');
# os.system('cp snorkel.db\ lftest_labels snorkel.db');

# %load_ext autoreload
# %autoreload 2

# from snorkel import SnorkelSession
# session = SnorkelSession()

# from snorkel.models import candidate_subclass

# Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

# from snorkel.models import CandidateSet
# train = session.query(CandidateSet).filter(
#     CandidateSet.name == 'Hardware Training Candidates').one()
# dev = session.query(CandidateSet).filter(
#     CandidateSet.name == 'Hardware Development Candidates').one()

# from snorkel.annotations import FeatureManager

# feature_manager = FeatureManager()

# %time F_train = feature_manager.create(session, train, 'Train Features')

# %time F_dev = feature_manager.update(session, dev, 'Train Features', expand_key_set=False)

# %time F_train = feature_manager.load(session, train, 'Train Features')

# from snorkel.utils import get_keys_by_candidate

# for f in get_keys_by_candidate(F_train, F_train.get_candidate(1))[:10]: print f
    
# import os
# os.system('cp snorkel.db snorkel.db\ lftest_featurized');

In [ ]:
# # If necessary:
# import os
# os.remove('snorkel.db');
# os.system('cp snorkel.db\ lftest_featurized snorkel.db');

# from snorkel import SnorkelSession
# session = SnorkelSession()

# from snorkel.models import candidate_subclass
# Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

# from snorkel.models import CandidateSet
# train = session.query(CandidateSet).filter(
#     CandidateSet.name == 'Hardware Training Candidates').one()

# from snorkel.annotations import LabelManager

# label_manager = LabelManager()


# from snorkel.lf_helpers import *

# LFs = []

# # gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
# # def LF_training_gold(c):
# #     training_gold_dict = get_gold_dict(gold_file, 'stg_temp_min')
# #     return 1 if ((c[0].parent.document.name).upper(), 
# #                  (c[0].get_span()).upper(), 
# #                  (''.join(c[1].get_span().split())).upper()) in training_gold_dict else 0
# # LFs.append(LF_training_gold)

# # from hardware_utils import load_extended_parts_dict
# # gold_file ='data/hardware/hardware_gold.csv'
# # parts_dict = load_extended_parts_dict(gold_file)
# # print "Loaded %d part numbers." % len(parts_dict)

# # def LF_parts_dict(c):
# #     return 1 if (c.temp.parent.words)[0] in parts_dict else -1
# # LFs.append(LF_foo)

# # POSITIVE

# def LF_to_right(c):
#     return 1 if 'to' in get_right_ngrams(c.temp, window=1, n_max=1) else 0
# LFs.append(LF_to_right)

# def LF_150_right(c):
#     return 1 if '150' in get_right_ngrams(c.temp, window=4, n_max=1) else 0
# LFs.append(LF_150_right)

# def LF_storage_row(c):
#     return 1 if 'storage' in get_row_ngrams(c.temp, n_max=2) else 0
# LFs.append(LF_storage_row)

# def LF_operating_row(c):
#     return 1 if 'operating' in get_row_ngrams(c.temp, n_max=2) else 0
# LFs.append(LF_operating_row)

# def LF_c_row(c):
#     return 1 if '%C' in get_row_ngrams(c.temp, n_min=1, n_max=2, case_sensitive=True) else 0
# LFs.append(LF_c_row)

# def LF_temperature_row(c):
#     return 1 if 'temperature' in get_row_ngrams(c.temp) else 0
# LFs.append(LF_temperature_row)

# def LF_tstg_row(c):
#     row_ngrams = get_row_ngrams(c.temp, n_max=1)
#     return 1 if ('tstg' in row_ngrams or 'ts' in row_ngrams) else 0
# LFs.append(LF_tstg_row)


# # NEGATIVE

# def LF_temp_outside_table(c):
#     return -1 if c.temp.parent.table is None else 0
# LFs.append(LF_temp_outside_table)

# def LF_mv_row(c):
#     return -1 if 'mv' in get_row_ngrams(c.temp, infer=True) else 0
# LFs.append(LF_mv_row)

# def LF_collector_row(c):
#     return -1 if 'collector' in get_row_ngrams(c.temp) else 0
# LFs.append(LF_collector_row)

# def LF_collectorattrib_row(c):
#     return -1 if 'collector%' in get_row_ngrams(c.temp) else 0
# LFs.append(LF_collectorattrib_row)

# def LF_typ_row(c):
#     return -1 if 'typ' in get_row_ngrams(c.temp) else 0
# LFs.append(LF_collector_row)

# def LF_complement_left(c):
#     return -1 if 'complement' in get_left_ngrams(c.part) else 0
# LFs.append(LF_complement_left)

# def LF_voltage_row(c):
#     return -1 if 'voltage' in get_row_ngrams(c.temp, infer=True) else 0
# LFs.append(LF_voltage_row)

# def LF_v_row(c):
#     return -1 if 'v' in get_row_ngrams(c.temp, infer=True) else 0
# LFs.append(LF_v_row)

# def LF_vceo_row(c):
#     return -1 if 'vceo' in get_row_ngrams(c.temp, infer=True) else 0
# LFs.append(LF_vceo_row)

# from snorkel.lf_helpers import get_right_ngrams, get_left_ngrams
# from snorkel.lf_helpers import contains_token, contains_regex
# from snorkel.lf_helpers import get_phrase_ngrams, get_cell_ngrams, get_neighbor_cell_ngrams
# from snorkel.lf_helpers import get_row_ngrams, get_col_ngrams, get_aligned_ngrams
# from snorkel.lf_helpers import same_document, same_table, same_cell, same_phrase
# c = train[12]
# print c.part
# print c.temp
# print c.temp.parent
# print c.temp.parent.table
# print same_document(c)
# print same_table(c)
# print contains_regex(c, rgx=r'849')
# print LF_temp_outside_table(c)

# %time L_train = label_manager.create(session, train, 'LF Labels', f=LFs)

# train_gold = session.query(CandidateSet).filter(
#     CandidateSet.name == 'Hardware Training Candidates -- Gold').one()

# %time print(L_train.lf_stats(train_gold))

# documents = set()
# for c in train:
#     documents.add(str(c.part.parent.document))
    
# from pprint import pprint
# pprint(documents)

# import os
# os.system('cp snorkel.db snorkel.db\ lftest_features');

In [ ]:
# import os
# os.remove('snorkel.db');
# os.system('cp snorkel.db\ lftest_features snorkel.db');

# %load_ext autoreload
# %autoreload 2

# from snorkel import SnorkelSession
# session = SnorkelSession()

# from snorkel.models import candidate_subclass

# Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

# from snorkel.models import CandidateSet
# train = session.query(CandidateSet).filter(
#     CandidateSet.name == 'Hardware Training Candidates').one()

# from snorkel.annotations import FeatureManager

# feature_manager = FeatureManager()

# %time F_train = feature_manager.load(session, train, 'Train Features')

# from snorkel.annotations import LabelManager

# label_manager = LabelManager()

# %time L_train = label_manager.load(session, train, 'LF Labels')

# from snorkel.learning import NaiveBayes

# gen_model = NaiveBayes()
# # gen_model.train(L_train, n_iter=3000, rate=1e-5)
# gen_model.train(L_train, n_iter=15000, rate=1e-2)

# gen_model.save(session, 'Generative Params')

# train_marginals = gen_model.marginals(L_train)

# from snorkel.learning import LogReg

# disc_model = LogReg()
# disc_model.train(F_train, train_marginals, n_iter=1000, rate=1e-3)

# disc_model.w.shape


# %time disc_model.save(session, "Discriminative Params")

# from snorkel.models import CandidateSet
# dev = session.query(CandidateSet).filter(
#     CandidateSet.name == 'Hardware Development Candidates').one()

# from snorkel.annotations import FeatureManager

# feature_manager = FeatureManager()
# %time F_dev = feature_manager.load(session, dev, 'Train Features')

# L_dev = label_manager.load(session, dev, "Hardware Development Labels -- Gold")

# gold_dev_set = session.query(CandidateSet).filter(
#     CandidateSet.name == 'Hardware Development Candidates -- Gold').one()

# F_dev.shape

# tp, fp, tn, fn = disc_model.score(F_dev, L_dev, gold_dev_set)

# from snorkel.models import Corpus
# import sys
# sys.path.append('../')
# from hardware_utils import entity_level_f1
# import os

# gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
# corpus = session.query(Corpus).filter(Corpus.name == 'Hardware Development').one()
# (TP, FP, FN) = entity_level_f1(tp, fp, tn, fn, gold_file, corpus, 'stg_temp_min')

# print FP